In [5]:
import pandas as pd
from scipy import stats
from itertools import combinations

In [6]:
df = pd.read_csv('Results_TD_CP.csv')

metrics = ['dti_fa', 'md', 'ad', 'rd']
groups = ['TD_ND', 'TD_D', 'CSC_LA', 'CSC_MA', 'PV_LA', 'PV_MA', 'Others_LA', 'Others_MA']

In [7]:
# Defining Strict Threshold (Bonferroni Correction)
# We have 8 groups, so there are 28 unique pairs.
# 0.05 / 28 = 0.00178. P-values must be lower than this to be significant.
# This prevents "false positives" from running so many tests.

pairs = list(combinations(groups, 2))
corrected_alpha = 0.05 / len(pairs)

In [10]:
print(f"Post-Hoc Analysis (Bonferroni Alpha: {corrected_alpha:.5f})")

for metric in metrics:
    print(f"\n {metric} Significant Pairs")
    subset = df[df['Metrics Name'] == metric]
    
    significant_found = False
    
    # Testing every possible pair of groups
    for group1, group2 in pairs:
        data1 = subset[group1].dropna().values
        data2 = subset[group2].dropna().values
        
        if len(data1) == 0 or len(data2) == 0:
            continue
            
        # Mann-Whitney U Test
        # 'two-sided' is the standard assumption
        stat, p_val = stats.mannwhitneyu(data1, data2, alternative='two-sided')
        
        if p_val < corrected_alpha:
            print(f"{group1} vs {group2} | p={p_val:.2e}")
            significant_found = True
            
    if not significant_found:
        print("No pairs met the strict significance threshold.")

Post-Hoc Analysis (Bonferroni Alpha: 0.00179)

 dti_fa Significant Pairs
TD_ND vs CSC_MA | p=2.97e-04
TD_D vs CSC_MA | p=1.04e-03

 md Significant Pairs
TD_ND vs PV_MA | p=3.50e-04

 ad Significant Pairs
TD_ND vs PV_LA | p=3.50e-04
TD_ND vs PV_MA | p=4.65e-05

 rd Significant Pairs
TD_ND vs CSC_MA | p=1.73e-04
TD_ND vs PV_MA | p=2.29e-05
TD_D vs PV_MA | p=1.69e-03
